<a href="https://colab.research.google.com/github/jejjohnson/gp_model_zoo/blob/master/code/numpyro/exact_gp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exact GP (MLE)

A demo notebook showing how solve a simple regression problem using an exact Gaussian Process (GP). This GP is trained with MLE and we will use the numpyro library as the backend for the inference.

In [ ]:
# #@title Install Packages
# %%capture
# !pip install pyprojroot jax jaxlib chex numpyro flax distrax numpy pandas seaborn matplotlib corner loguru nb_black sklearn

In [ ]:
import sys, os
from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".root"])

# append to path
sys.path.append(str(root))

In [ ]:
# @title Load Packages
# TYPE HINTS
from typing import Tuple, Optional, Dict, Callable, Union

# JAX SETTINGS
import jax
import jax.numpy as jnp
import jax.random as random
from jax.config import config

config.update("jax_enable_x64", True)


# JAX UTILITY LIBRARIES
import chex

# NUMPYRO SETTINGS
import numpyro
import numpyro.distributions as dist
from numpyro.infer.autoguide import AutoDiagonalNormal
from numpyro.infer import SVI, Trace_ELBO

# NUMPY SETTINGS
import numpy as np

np.set_printoptions(precision=3, suppress=True)

# MATPLOTLIB Settings
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# SEABORN SETTINGS
import seaborn as sns

sns.set_context(context="poster", font_scale=0.8)

# PANDAS SETTINGS
import pandas as pd

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

# LOGGING SETTINGS
import loguru

%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data import regression_complex, regression_simple

Xtrain, ytrain, xtest, ytest = regression_simple(
    n_train=250, n_test=1_000, noise=0.3, seed=123, buffer=0.3
)

fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(Xtrain, ytrain, color="tab:orange", label="Data")
ax.plot(xtest, ytest, color="black", label="True Function")
ax.legend()
plt.tight_layout()
plt.show()

## Gaussian Process Model

#### Kernel

In [ ]:
from chex import Array
from src.utils import identity_matrix, add_to_diagonal
from src.kernels import RBF

In [ ]:
rbf_kernel = RBF(length_scale=1.0, variance=1.0)
K = rbf_kernel.gram(Xtrain)

# check shape
chex.assert_shape(K, (Xtrain.shape[0], Xtrain.shape[0]))

In [ ]:
fig, ax = plt.subplots()

ax.imshow(K, interpolation="nearest")
plt.tight_layout()
plt.show()

### Model

In [ ]:
from src.exact import GPRModel
from src.means import zero_mean

jitter = 1e-5


def numpyro_model(X, y):

    #     # Set priors on hyperparameters.
    #     η = numpyro.sample("variance", dist.HalfCauchy(scale=5.0))
    #     ℓ = numpyro.sample("length_scale", dist.Gamma(2.0, 1.0))
    #     σ = numpyro.sample("obs_noise", dist.HalfCauchy(scale=5.0))

    # set params and constraints on hyperparams
    η = numpyro.param("variance", init_value=1.0, constraints=dist.constraints.positive)
    ℓ = numpyro.param(
        "length_scale", init_value=0.1, constraints=dist.constraints.positive
    )
    σ = numpyro.param("obs_noise", init_value=0.1, onstraints=dist.constraints.positive)

    # Kernel Function
    rbf_kernel = RBF(variance=η, length_scale=ℓ)

    # GP Model
    gp_model = GPRModel(
        X=X, y=y, mean=zero_mean, kernel=rbf_kernel, obs_noise=σ, jitter=jitter
    )

    # Sample y according SGP
    return gp_model.to_numpyro(y=y)


def empty_guide(X, y):
    pass

In [ ]:
with numpyro.handlers.seed(rng_seed=123):
    t = numpyro_model(Xtrain, ytrain)

assert t.shape == Xtrain.shape

## Training (Maximum Likelihood)

In [ ]:
from numpyro.infer.autoguide import (
    AutoDelta,
)  # AutoDiagonalNormal, AutoIAFNormal, AutoLaplaceApproximation, AutoMultivariateNormal

In [ ]:
# ===================
# Model
# ===================
# GP model

# delta guide - basically deterministic
delta_guide = AutoDelta(numpyro_model)
mll_guide = empty_guide

In [ ]:
# reproducibility
rng_key = random.PRNGKey(0)


# Setup
optimizer = numpyro.optim.Adam(step_size=0.01)
# optimizer = numpyro.optim.Minimize()
# optimizer = optax.adamw(learning_rate=0.1)
svi = SVI(numpyro_model, mll_guide, optimizer, loss=Trace_ELBO())
svi_results = svi.run(random.PRNGKey(1), 100, Xtrain, ytrain.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(svi_results.losses)
ax.set(title="Loss", xlabel="Iterations", ylabel="Negative Log-Likelihood")
plt.show()

### Extracting the Params

In [ ]:
from pprint import pprint

# Take them directly
learned_params = svi_results.params
# x_u = learned_params["x_u"]
# learned_params = delta_guide.median(learned_params)
# learned_params["x_u"] = x_u

### Predictions

In [ ]:
from src.exact import init_gp_predictive, get_cond_params

gp_pred = init_gp_predictive(RBF, learned_params, Xtrain, ytrain, jitter=1e-5)

In [ ]:
gp_pred_jitted = jax.jit(jax.partial(gp_pred.predict_y, full_covariance=False))

mu, var = gp_pred_jitted(xtest)

std = jnp.sqrt(var.squeeze())

In [ ]:
%timeit gp_pred_jitted(jnp.array(xtest).block_until_ready())
%timeit gp_pred.predict_y(jnp.array(xtest).block_until_ready())

In [ ]:
ci = 1.96
one_stddev = ci * std

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(Xtrain, ytrain.squeeze(), "o", color="tab:orange", label="Training Data")
ax.plot(xtest, ytest, color="black", linestyle="-", label="True Function")
ax.plot(
    xtest,
    mu.ravel(),
    color="Blue",
    linestyle="--",
    linewidth=3,
    label="Predictive Mean",
)
ax.fill_between(
    xtest.ravel(),
    mu.ravel() - one_stddev,
    mu.ravel() + one_stddev,
    alpha=0.4,
    color="tab:blue",
    label=f" 95% Confidence Interval",
)
ax.plot(xtest, mu.ravel() - one_stddev, linestyle="--", color="tab:blue")
ax.plot(xtest, mu.ravel() + one_stddev, linestyle="--", color="tab:blue")
plt.tight_layout()
plt.legend(fontsize=12)
plt.show()

## Calibration

In [ ]:
from uncertainty_toolbox import viz as utviz

#### Training Data

In [ ]:
mu, var = gp_pred.predict_y(Xtrain)

std = jnp.sqrt(var.squeeze())

utviz.plot_calibration(y_pred=mu.ravel(), y_std=std.ravel(), y_true=ytrain.ravel())

#### Test Data

In [ ]:
mu, var = gp_pred.predict_y(xtest)

std = jnp.sqrt(var.squeeze())

utviz.plot_calibration(y_pred=mu.ravel(), y_std=std.ravel(), y_true=ytest.ravel())